# Imports

In [238]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import gensim.downloader as api

# Data Loading

In [239]:
## Load cleaned data
X_reviews = pd.read_csv("/Users/egmac/code/arostagnat/BookMatch/data/proc_data/cluster_result/X_bert_cluster_150.csv")

In [240]:
## Load metadata
metadata_movies = pd.read_json("/Users/egmac/code/arostagnat/BookMatch/data/raw_data/raw_movies/metadata.json", lines=True)
metadata_books = pd.read_json("/Users/egmac/code/arostagnat/BookMatch/data/raw_data/raw_book/metadata.json", lines=True)

In [241]:
# Adjust metadata columns to match X_reviews
metadata_movies.rename({"item_id":"item_id_movie", "title":"title_movie"}, axis='columns',inplace=True)
metadata_books.rename({"item_id":"item_id_book", "title":"title_book","img":"img_book","url":"url_book"}, axis='columns',inplace=True)

In [242]:
## Adjust import to replace $$$ with 0, and convert item_id to float to enable merge. 
## Note that the X_reviews import is preformatted as a float
# X_reviews = X_reviews.replace({'$$$': 0}, regex=False)
X_reviews.item_id_movie = X_reviews.item_id_movie.astype(float)
X_reviews.item_id_book = X_reviews.item_id_book.astype(float)

In [243]:
## Revise metadata item_id to float to match X_reviews
metadata_movies.item_id_movie = metadata_movies.item_id_movie.astype(float)
metadata_books.item_id_book = metadata_books.item_id_book.astype(float)

In [244]:
## Merge X_reviews and metadata
X_all = pd.merge(X_reviews, metadata_movies[["title_movie","item_id_movie"]], on="item_id_movie", how="left")
X_all = pd.merge(X_all, metadata_books[["title_book","item_id_book","url_book","img_book"]], on="item_id_book", how="left")

In [245]:
## Check import
X_all.head(1)

,item_id_movie,is_movie,item_id_book,clustering_label_bert,vector,title_movie,title_book,url_book,img_book
0,132692.0,1.0,-1.0,0,[-1.79571323e-02 3.01178787e-02 -2.63748504e-...,Frontier Rangers (1959),NaN,NaN,NaN


In [246]:
X_all.shape

(27532, 9)

In [247]:
## Import book ratings for recommendation filtering
ratings_books =  pd.read_json("/Users/egmac/code/arostagnat/BookMatch/data/raw_data/raw_book/ratings.json", lines=True)
ratings_books = ratings_books.rename(columns={"item_id":"item_id_book"})

In [248]:
## Calculate average rating for each book
avg_ratings_books = ratings_books.groupby(["item_id_book"]).mean().drop(columns=["user_id"])
print(f"Original ratings df: {ratings_books.shape} | Average ratings df: {avg_ratings_books.shape}")

Original ratings df: (5152656, 3) | Average ratings df: (9374, 1)


# Post-processing

In [249]:
## Import relevant packages

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [250]:
## Create extract of X_all with vectors spread across columns, and confirm relevance of dimensions
## Vectors have to be reformatted as lists, as they are formatted as strings with ""

vectors = X_all.vector.tolist()
vectors_revised = []

for vector in vectors:
    result = vector.strip('[]').replace("'","").replace("\n","").split()
    result = [float(i) for i in result]
    vectors_revised.append(result)

X_vectors = pd.DataFrame(vectors_revised)

In [251]:
# svd = TruncatedSVD(n_components=X_vectors.shape[1])
# svd_result = svd.fit_transform(X_vectors)

In [252]:
## Plot variance as a function of the number of components.
## Based on the below figure, nearly 100% of the variance is explained by 250 components
# plt.plot(svd.explained_variance_ratio_.cumsum())
# plt.xlabel('Number of singular value components')
# plt.ylabel('Cumulative percent of variance')   
# plt.grid()
# plt.show()

In [253]:
## Reshape vectors to 250 components, which will help reduce computational time
# n = 250
# X_vectors_revised = pd.DataFrame(X_vectors.iloc[:,0:n])
# print(f'X_all shape: {X_all.shape} | X_vectors shape: {X_vectors.shape}')

In [254]:
## Add qualitative columns to X_vectors_revised, and then create 2 separate dataframes for books and movies.
## Movie dataframe to be used to lookup user-inputted movies. Book dataframe to be used for calculations.
## Note that the dataframes need to be separated eventually, so it's worth doing now.

X_vectors[["item_id_movie","item_id_book","is_movie","clustering_label_bert"]] = X_all[["item_id_movie","item_id_book","is_movie","clustering_label_bert"]]
X_vectors_movies = X_vectors[X_vectors.is_movie == 1].set_index("item_id_movie",drop=True).drop(columns=["item_id_book","is_movie"])
X_vectors_books = X_vectors[X_vectors.is_movie == 0].set_index("item_id_book",drop=True).drop(columns=["item_id_movie","is_movie"])

In [255]:
X_vectors_good_books = pd.merge(X_vectors_books,avg_ratings_books,how="left",on="item_id_book")
X_vectors_good_books = X_vectors_good_books[X_vectors_good_books.rating >= 4].drop(columns=["rating"])
X_vectors_good_books.head(2)

/var/folders/_c/hnby49996jgf113yx9b06pd80000gn/T/ipykernel_32289/3642167841.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_vectors_good_books = pd.merge(X_vectors_books,avg_ratings_books,how="left",on="item_id_book")


,0,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,clustering_label_bert
item_id_book,,,,,,,,,,,,,,,,,,,,,
45524554.0,-0.047604,0.034708,-0.042140,0.023047,0.029068,-0.003054,0.122946,0.034663,0.016583,0.012794,...,-0.076962,0.074651,0.008275,-0.015889,0.030595,0.049729,-0.015802,0.001648,-0.044022,1
44707128.0,-0.042919,-0.050687,0.070148,-0.017286,-0.005120,0.042632,0.030215,-0.024918,0.116753,-0.010577,...,-0.010735,-0.023382,0.011517,0.038426,0.049266,-0.036537,-0.039882,-0.009105,-0.108453,2


In [256]:
print(f"X_vectors_books: {X_vectors_books.shape} | X_vectors_books.ratings:{X_vectors_good_books.shape}")

X_vectors_books: (5076, 385) | X_vectors_books.ratings:(2149, 385)


# Recommendations

In [257]:
## Method 1: For each film in user list, calculate the cosine similarity with all books in the cluster
## Then, sort the books by their cosine similarity to identify **the** **closest** book for each film
## Finally, take the full list of book recommendations and then identify the **top 5 most frequent** books

def get_local_reccs(user_movies:list):
    
    verified_movies = [movie_id for movie_id in user_movies if movie_id in X_all.item_id_movie.tolist()]
    
    recommendations = pd.DataFrame(columns=["similarity","title_book","img_book","url_book"])
    movies = pd.DataFrame(verified_movies,columns=["item_id_movie"])
    movies = pd.merge(movies,X_all[["title_movie","item_id_movie"]],on="item_id_movie",how="left")
    
    for movie_id in verified_movies:

        # Obtain vectors for user-inputted film and all books. Clusters are not used for time being
        ### movie_cluster = X_vectors_movies[X_vectors_movies.index == movie_id].cluster_bert.values[0]
        movie_vector = X_vectors_movies[X_vectors_movies.index == movie_id].drop(columns=["clustering_label_bert"])
        books_vectors = X_vectors_books.drop(columns=["clustering_label_bert"])

        # Calculate cosine similarity
        sim_books = cosine_similarity(books_vectors,movie_vector)
 
        # Create summary table of books with their similarity and relevant details
        sim_books_detail = pd.DataFrame(sim_books,index=books_vectors.index,columns=["similarity"])
        sim_books_detail = sim_books_detail.sort_values("similarity",ascending=False)
        sim_books_detail = pd.merge(sim_books_detail,X_all[["title_book","img_book","url_book","item_id_book"]],
                                    on="item_id_book",how="left")

        # Add top book to recommendations dataframe
        top_book = pd.DataFrame([sim_books_detail.loc[0]])
        recommendations = pd.concat([recommendations,top_book],axis=0, ignore_index=True)
    
    print("Inputted films")
    print(movies)
    return recommendations["title_book"]


In [258]:
## Method 2: Calculate the average vector for all films in the user list, and then identify the corresponding cluster
## Then,calculate the cosine similarity with all books in the cluster
## Finally, sort the books by their cosine similarities and take the **top 5 closest** books

def get_global_reccs(user_movies:list):
    
    verified_movies = [movie_id for movie_id in user_movies if movie_id in X_all.item_id_movie.tolist()]
    
    ## Collect vectors of all inputted films and calculate average vector
    movies_id = pd.DataFrame(verified_movies,columns=["item_id_movie"])
    movies_vectors = pd.merge(movies_id,
                              X_vectors_movies,
                              how="left",
                              on="item_id_movie").set_index("item_id_movie").drop(columns=["clustering_label_bert"])
    avg_movie_vector = pd.DataFrame([movies_vectors.mean(numeric_only=True)])
    books_vectors = X_vectors_books.drop(columns=["clustering_label_bert"])

    ## Calculate cosine similarity
    sim_books = cosine_similarity(books_vectors,avg_movie_vector)

    ## Create summary table of books with their similarity and relevant details
    sim_books_detail = pd.DataFrame(sim_books,index=books_vectors.index,columns=["similarity"])
    sim_books_detail = sim_books_detail.sort_values("similarity",ascending=False)
    sim_books_detail = pd.merge(sim_books_detail,X_all[["title_book","img_book","url_book","item_id_book"]],on="item_id_book",how="left")
    
    ## Take top 5 books and show results
    recommendations = sim_books_detail.head(5)
    movie_titles = pd.merge(movies_id,X_all[["title_movie","item_id_movie"]],how="inner",on="item_id_movie")
    print("Inputted films")
    print(movie_titles.title_movie)
    print ("Top 5 book recommendations")
    return recommendations["title_book"]

# Recommendations with ratings

In [259]:
def get_local_reccs_rating (user_movies:list):
    
    verified_movies = [movie_id for movie_id in user_movies if movie_id in X_all.item_id_movie.tolist()]
    
    recommendations = pd.DataFrame(columns=["similarity","title_book","img_book","url_book"])
    movies = pd.DataFrame(verified_movies,columns=["item_id_movie"])
    movies = pd.merge(movies,X_all[["title_movie","item_id_movie"]],on="item_id_movie",how="left")
    
    for movie_id in verified_movies:

        # Obtain vectors for user-inputted film and all books. Clusters are not used for time being
        movie_vector = X_vectors_movies[X_vectors_movies.index == movie_id].drop(columns=["clustering_label_bert"])
        books_vectors = X_vectors_good_books.drop(columns=["clustering_label_bert"])

        # Calculate cosine similarity
        sim_books = cosine_similarity(books_vectors,movie_vector)
 
        # Create summary table of books with their similarity and relevant details
        sim_books_detail = pd.DataFrame(sim_books,index=books_vectors.index,columns=["similarity"])
        sim_books_detail = sim_books_detail.sort_values("similarity",ascending=False)
        sim_books_detail = pd.merge(sim_books_detail,X_all[["title_book","img_book","url_book","item_id_book"]],
                                    on="item_id_book",how="left")

        # Add top book to recommendations dataframe
        top_book = pd.DataFrame([sim_books_detail.loc[0]])
        recommendations = pd.concat([recommendations,top_book],axis=0, ignore_index=True)
    
    print("Inputted films")
    print(movies)
    return recommendations["title_book"]


In [260]:
def get_global_reccs_rating (user_movies:list):
    
    verified_movies = [movie_id for movie_id in user_movies if movie_id in X_all.item_id_movie.tolist()]
    
    ## Collect vectors of all inputted films and calculate average vector
    movies_id = pd.DataFrame(verified_movies,columns=["item_id_movie"])
    movies_vectors = pd.merge(movies_id,
                              X_vectors_movies,
                              how="left",
                              on="item_id_movie").set_index("item_id_movie").drop(columns=["clustering_label_bert"])
    avg_movie_vector = pd.DataFrame([movies_vectors.mean(numeric_only=True)])
    books_vectors = X_vectors_good_books.drop(columns=["clustering_label_bert"])

    ## Calculate cosine similarity
    sim_books = cosine_similarity(books_vectors,avg_movie_vector)

    ## Create summary table of books with their similarity and relevant details
    sim_books_detail = pd.DataFrame(sim_books,index=books_vectors.index,columns=["similarity"])
    sim_books_detail = sim_books_detail.sort_values("similarity",ascending=False)
    sim_books_detail = pd.merge(sim_books_detail,X_all[["title_book","img_book","url_book","item_id_book"]],on="item_id_book",how="left")
    
    ## Take top 5 books and show results
    recommendations = sim_books_detail.head(5)
    movie_titles = pd.merge(movies_id,X_all[["title_movie","item_id_movie"]],how="inner",on="item_id_movie")
    print("Inputted films")
    print(movie_titles.title_movie)
    print ("Top 5 book recommendations")
    return recommendations["title_book"]

# Recommendations with clustering

In [261]:
def get_local_reccs_cluster(user_movies:list):
    
    verified_movies = [movie_id for movie_id in user_movies if movie_id in X_all.item_id_movie.tolist()]
    
    recommendations = pd.DataFrame(columns=["similarity","title_book","img_book","url_book"])
    movies = pd.DataFrame(verified_movies,columns=["item_id_movie"])
    movies = pd.merge(movies,X_all[["title_movie","item_id_movie"]],on="item_id_movie",how="left")
    
    for movie_id in verified_movies:

        # Obtain vectors for user-inputted film and all books.
        movie_cluster = X_vectors_movies[X_vectors_movies.index == movie_id].clustering_label_bert.values[0]
        movie_vector = X_vectors_movies[X_vectors_movies.index == movie_id].drop(columns=["clustering_label_bert"])
        books_vectors = X_vectors_books[X_vectors_books.clustering_label_bert == movie_cluster].drop(columns=["clustering_label_bert"])

        # Calculate cosine similarity
        sim_books = cosine_similarity(books_vectors,movie_vector)
 
        # Create summary table of books with their similarity and relevant details
        sim_books_detail = pd.DataFrame(sim_books,index=books_vectors.index,columns=["similarity"])
        sim_books_detail = sim_books_detail.sort_values("similarity",ascending=False)
        sim_books_detail = pd.merge(sim_books_detail,X_all[["title_book","img_book","url_book","item_id_book"]],
                                    on="item_id_book",how="left")

        # Add top book to recommendations dataframe
        top_book = pd.DataFrame([sim_books_detail.loc[0]])
        recommendations = pd.concat([recommendations,top_book],axis=0, ignore_index=True)
    
    print("Inputted films")
    print(movies)
    return recommendations["title_book"]


In [262]:
def get_global_reccs_cluster(user_movies:list):
    
    verified_movies = [movie_id for movie_id in user_movies if movie_id in X_all.item_id_movie.tolist()]
    
    ## Collect vectors of all inputted films and calculate average vector
    movies_id = pd.DataFrame(verified_movies,columns=["item_id_movie"])
    movies_vectors = pd.merge(movies_id,
                              X_vectors_movies,
                              how="left",
                              on="item_id_movie").set_index("item_id_movie")
    avg_movie_vector = pd.DataFrame([movies_vectors.mean(numeric_only=True)]).drop(columns=["clustering_label_bert"])
    all_movies_vectors = X_vectors_movies.drop(columns=["clustering_label_bert"])
    
    ## Find cluster of nearest item (film)
    sim_movies = cosine_similarity(all_movies_vectors,avg_movie_vector)
    sim_movies_detail = pd.DataFrame(sim_movies,
                                     index=all_movies_vectors.index,
                                     columns=["similarity"]).sort_values("similarity",ascending=False).reset_index()
    closest_movie_id = sim_movies_detail.loc[0].item_id_movie
    closest_cluster = X_vectors_movies[X_vectors_movies.index == closest_movie_id].clustering_label_bert.values[0]
    books_vectors = X_vectors_books[X_vectors_books.clustering_label_bert== closest_cluster].drop(columns=["clustering_label_bert"])

    ## Calculate cosine similarity
    sim_books = cosine_similarity(books_vectors,avg_movie_vector)

    ## Create summary table of books with their similarity and relevant details
    sim_books_detail = pd.DataFrame(sim_books,index=books_vectors.index,columns=["similarity"])
    sim_books_detail = sim_books_detail.sort_values("similarity",ascending=False)
    sim_books_detail = pd.merge(sim_books_detail,X_all[["title_book","img_book","url_book","item_id_book"]],on="item_id_book",how="left")
    
    ## Take top 5 books and show results
    recommendations = sim_books_detail.head(5)
    movie_titles = pd.merge(movies_id,X_all[["title_movie","item_id_movie"]],how="inner",on="item_id_movie")
    print("Inputted films")
    print(movie_titles.title_movie)
    print ("Top 5 book recommendations")
    return recommendations["title_book"]

In [263]:
user_movies = [1,1,3,4,5,6,7,8]
verified_movies = [movie_id for movie_id in user_movies if movie_id in X_all.item_id_movie.tolist()]

## Collect clusters of all inputted films and identify most frequent cluster
movies_id = pd.DataFrame(verified_movies,columns=["item_id_movie"])
movies_vectors = pd.merge(movies_id,
                          X_vectors_movies,
                          how="left",
                          on="item_id_movie").set_index("item_id_movie")
movies_clusters = movies_vectors.groupby(["clustering_label_bert"]).count()[0].reset_index().rename(columns={
    0:"cluster_count"})
movies_clusters = movies_clusters.sort_values(by="cluster_count",ascending=False).reset_index().drop(columns=["index"])
counts = movies_clusters.cluster_count.to_list()
avg_cluster = movies_clusters.loc[0]["clustering_label_bert"]
avg_movie_vector = pd.DataFrame([movies_vectors[movies_vectors.clustering_label_bert == main_cluster].mean(
            numeric_only=True)]).drop(columns=["clustering_label_bert"])
avg_movie_vector

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,-0.021117,-0.031267,0.077505,0.017031,-0.013605,0.02245,0.071607,0.057103,0.003765,0.070925,...,0.061121,-0.022265,-0.011456,0.040391,-0.025528,-0.032638,0.033434,0.007523,0.05141,0.039787


In [264]:
def get_freq_cluster_reccs(user_movies:list):
    verified_movies = [movie_id for movie_id in user_movies if movie_id in X_all.item_id_movie.tolist()]
    
    ## Collect clusters of all inputted films and identify most frequent cluster
    movies_id = pd.DataFrame(verified_movies,columns=["item_id_movie"])
    movies_vectors = pd.merge(movies_id,
                              X_vectors_movies,
                              how="left",
                              on="item_id_movie").set_index("item_id_movie")
    
    movies_clusters = movies_vectors.groupby(["clustering_label_bert"]).count()[0].reset_index().rename(columns={
    0:"cluster_count"})
    movies_clusters = movies_clusters.sort_values(by="cluster_count",ascending=False).reset_index().drop(columns=["index"])
    movies_clusters_counts = movies_clusters.cluster_count.to_list()
    main_cluster = movies_clusters.loc[0]["clustering_label_bert"]
    
    if main_cluster > 1:
        
        avg_movie_vector = pd.DataFrame([movies_vectors[movies_vectors.clustering_label_bert == main_cluster].mean(
            numeric_only=True)]).drop(columns=["clustering_label_bert"])
        
#         books_vectors = X_vectors_books[X_vectors_books.clustering_label_bert== main_cluster].drop(columns=["clustering_label_bert"])
        books_vectors = X_vectors_books.drop(columns=["clustering_label_bert"])

        sim_books = cosine_similarity(books_vectors,avg_movie_vector)
        
        sim_books_detail = pd.DataFrame(sim_books,index=books_vectors.index,columns=["similarity"])
        sim_books_detail = sim_books_detail.sort_values("similarity",ascending=False)
        sim_books_detail = pd.merge(sim_books_detail,X_all[["title_book","img_book","url_book","item_id_book"]],on="item_id_book",how="left")

        ## Take top 5 books and show results
        recommendations = sim_books_detail.head(5)
        movie_titles = pd.merge(movies_id,X_all[["title_movie","item_id_movie"]],how="inner",on="item_id_movie")
        print("Inputted films")
        print(movie_titles.title_movie)
        print(movies_clusters)
        print ("Top 5 book recommendations")
        return recommendations["title_book"]
        
    else:
        
        avg_movie_vector = pd.DataFrame([movies_vectors.mean(numeric_only=True)]).drop(columns=["clustering_label_bert"])

#         all_movies_vectors = X_vectors_movies.drop(columns=["clustering_label_bert"])
#         sim_movies = cosine_similarity(all_movies_vectors,avg_movie_vector)
#         sim_movies_detail = pd.DataFrame(sim_movies,
#                                          index=all_movies_vectors.index,
#                                          columns=["similarity"]).sort_values("similarity",ascending=False).reset_index()
#         closest_movie_id = sim_movies_detail.loc[0].item_id_movie
#         closest_cluster = X_vectors_movies[X_vectors_movies.index == closest_movie_id].clustering_label_bert.values[0]
        books_vectors = X_vectors_books.drop(columns=["clustering_label_bert"])

        ## Calculate cosine similarity
        sim_books = cosine_similarity(books_vectors,avg_movie_vector)

        ## Create summary table of books with their similarity and relevant details
        sim_books_detail = pd.DataFrame(sim_books,index=books_vectors.index,columns=["similarity"])
        sim_books_detail = sim_books_detail.sort_values("similarity",ascending=False)
        sim_books_detail = pd.merge(sim_books_detail,X_all[["title_book","img_book","url_book","item_id_book"]],on="item_id_book",how="left")

        ## Take top 5 books and show results
        recommendations = sim_books_detail.head(5)
        movie_titles = pd.merge(movies_id,X_all[["title_movie","item_id_movie"]],how="inner",on="item_id_movie")
        print("Inputted films")
        print(movie_titles.title_movie)
        print(movies_clusters)
        print ("Top 5 book recommendations")
        return recommendations["title_book"]

# Recommendations with clusters and ratings

In [265]:
def get_local_reccs_cluster_rating(user_movies:list):
    
    verified_movies = [movie_id for movie_id in user_movies if movie_id in X_all.item_id_movie.tolist()]
    
    recommendations = pd.DataFrame(columns=["similarity","title_book","img_book","url_book"])
    movies = pd.DataFrame(verified_movies,columns=["item_id_movie"])
    movies = pd.merge(movies,X_all[["title_movie","item_id_movie"]],on="item_id_movie",how="left")
    
    for movie_id in verified_movies:

        # Obtain vectors for user-inputted film and all books.
        movie_cluster = X_vectors_movies[X_vectors_movies.index == movie_id].clustering_label_bert.values[0]
        movie_vector = X_vectors_movies[X_vectors_movies.index == movie_id].drop(columns=["clustering_label_bert"])
        books_vectors = X_vectors_good_books[X_vectors_books.clustering_label_bert == movie_cluster].drop(columns=["clustering_label_bert"])

        # Calculate cosine similarity
        sim_books = cosine_similarity(books_vectors,movie_vector)
 
        # Create summary table of books with their similarity and relevant details
        sim_books_detail = pd.DataFrame(sim_books,index=books_vectors.index,columns=["similarity"])
        sim_books_detail = sim_books_detail.sort_values("similarity",ascending=False)
        sim_books_detail = pd.merge(sim_books_detail,X_all[["title_book","img_book","url_book","item_id_book"]],
                                    on="item_id_book",how="left")

        # Add top book to recommendations dataframe
        top_book = pd.DataFrame([sim_books_detail.loc[0]])
        recommendations = pd.concat([recommendations,top_book],axis=0, ignore_index=True)
    
    print("Inputted films")
    print(movies)
    return recommendations["title_book"]


In [266]:
def get_global_reccs_cluster_rating(user_movies:list):
    
    verified_movies = [movie_id for movie_id in user_movies if movie_id in X_all.item_id_movie.tolist()]
    
    ## Collect vectors of all inputted films and calculate average vector
    movies_id = pd.DataFrame(verified_movies,columns=["item_id_movie"])
    movies_vectors = pd.merge(movies_id,
                              X_vectors_movies,
                              how="left",
                              on="item_id_movie").set_index("item_id_movie")
    avg_movie_vector = pd.DataFrame([movies_vectors.mean(numeric_only=True)]).drop(columns=["clustering_label_bert"])
    all_movies_vectors = X_vectors_movies.drop(columns=["clustering_label_bert"])
    
    ## Find cluster of nearest item (film)
    sim_movies = cosine_similarity(all_movies_vectors,avg_movie_vector)
    sim_movies_detail = pd.DataFrame(sim_movies,
                                     index=all_movies_vectors.index,
                                     columns=["similarity"]).sort_values("similarity",ascending=False).reset_index()
    closest_movie_id = sim_movies_detail.loc[0].item_id_movie
    closest_cluster = X_vectors_movies[X_vectors_movies.index == closest_movie_id].clustering_label_bert.values[0]
    books_vectors = X_vectors_good_books[X_vectors_goodb_books.clustering_label_bert== closest_cluster].drop(columns=["clustering_label_bert"])

    ## Calculate cosine similarity
    sim_books = cosine_similarity(books_vectors,avg_movie_vector)

    ## Create summary table of books with their similarity and relevant details
    sim_books_detail = pd.DataFrame(sim_books,index=books_vectors.index,columns=["similarity"])
    sim_books_detail = sim_books_detail.sort_values("similarity",ascending=False)
    sim_books_detail = pd.merge(sim_books_detail,X_all[["title_book","img_book","url_book","item_id_book"]],on="item_id_book",how="left")
    
    ## Take top 5 books and show results
    recommendations = sim_books_detail.head(5)
    movie_titles = pd.merge(movies_id,X_all[["title_movie","item_id_movie"]],how="inner",on="item_id_movie")
    print("Inputted films")
    print(movie_titles.title_movie)
    print ("Top 5 book recommendations")
    return recommendations["title_book"]

# Illustrative results

In [267]:
# films = ["Harry Potter and the Goblet of Fire (2005)",
#          "Maze Runner, The (2014)",
#          "Harry Potter and the Deathly Hallows: Part 2 (2011)"]

films = X_all.title_movie.dropna().to_list()
harry_potter_movies = [film for film in films if "Potter" in film]
harry_potter_movies

["Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
 'Harry Potter and the Goblet of Fire (2005)',
 'Private Potter (1962)',
 'Harry Potter and the Deathly Hallows: Part 2 (2011)',
 'Very Potter Musical, A (2009)']

In [311]:
cluster_movies = [film for film in films if "prince" in film]
cluster_movies

['Ali Zoua: Prince of the Streets (Ali Zaoua, prince de la rue) (2000)',
 'Princess of Montpensier, The (La princesse de Montpensier) (2010)']

In [269]:
suggested_film_ids = []

for film in harry_potter_movies:
    if film in X_all.title_movie.tolist():
        movie_id = X_all[X_all.title_movie == film].item_id_movie.values[0]
        suggested_film_ids.append(movie_id)

In [270]:
def get_movie_ids(movie_titles):
    movie_ids = []
    for movie in movie_titles:
        if movie in X_all.title_movie.tolist():
            movie_id = X_all[X_all.title_movie == movie].item_id_movie.values[0]
            movie_ids.append(movie_id)
    return movie_ids

In [271]:
cluster_film_ids = get_movie_ids(cluster_movies)

### Local apporach

In [272]:
get_local_reccs(suggested_film_ids)

Inputted films
   item_id_movie                                        title_movie
0         4896.0  Harry Potter and the Sorcerer's Stone (a.k.a. ...
1        40815.0         Harry Potter and the Goblet of Fire (2005)
2       150330.0                              Private Potter (1962)
3        88125.0  Harry Potter and the Deathly Hallows: Part 2 (...
4        93006.0                      Very Potter Musical, A (2009)


0                     The Magicians (The Magicians #1)
1    Harry Potter and the Prisoner of Azkaban (Harr...
2                                         Redeployment
3               Frost Like Night (Snow Like Ashes, #3)
4             The Walking Dead, Vol. 01: Days Gone Bye
Name: title_book, dtype: object

In [273]:
get_local_reccs_rating(suggested_film_ids)

Inputted films
   item_id_movie                                        title_movie
0         4896.0  Harry Potter and the Sorcerer's Stone (a.k.a. ...
1        40815.0         Harry Potter and the Goblet of Fire (2005)
2       150330.0                              Private Potter (1962)
3        88125.0  Harry Potter and the Deathly Hallows: Part 2 (...
4        93006.0                      Very Potter Musical, A (2009)


0    Harry Potter and the Order of the Phoenix (Har...
1    Harry Potter and the Prisoner of Azkaban (Harr...
2                                         Redeployment
3    Harry Potter and the Sorcerer's Stone (Harry P...
4             The Walking Dead, Vol. 01: Days Gone Bye
Name: title_book, dtype: object

In [274]:
get_local_reccs_cluster(suggested_film_ids)

Inputted films
   item_id_movie                                        title_movie
0         4896.0  Harry Potter and the Sorcerer's Stone (a.k.a. ...
1        40815.0         Harry Potter and the Goblet of Fire (2005)
2       150330.0                              Private Potter (1962)
3        88125.0  Harry Potter and the Deathly Hallows: Part 2 (...
4        93006.0                      Very Potter Musical, A (2009)


0                           My Man Jeeves (Jeeves, #1)
1    Harry Potter and the Prisoner of Azkaban (Harr...
2                   It's in His Kiss (Bridgertons, #7)
3    Harry Potter and the Sorcerer's Stone (Harry P...
4                      Rock Hard (Sinners on Tour, #2)
Name: title_book, dtype: object

In [275]:
# get_local_reccs_cluster_rating([1,2,3,4,5,6,7])

### Global approach

In [276]:
get_global_reccs(suggested_film_ids)

Inputted films
0    Harry Potter and the Sorcerer's Stone (a.k.a. ...
1           Harry Potter and the Goblet of Fire (2005)
2                                Private Potter (1962)
3    Harry Potter and the Deathly Hallows: Part 2 (...
4                        Very Potter Musical, A (2009)
Name: title_movie, dtype: object
Top 5 book recommendations


0    Fantastic Beasts and Where to Find Them: The O...
1    Harry Potter and the Cursed Child - Parts One ...
2    Harry Potter and the Order of the Phoenix (Har...
3    Harry Potter and the Prisoner of Azkaban (Harr...
4              The Magician's Land (The Magicians, #3)
Name: title_book, dtype: object

In [277]:
get_global_reccs_rating(suggested_film_ids)

Inputted films
0    Harry Potter and the Sorcerer's Stone (a.k.a. ...
1           Harry Potter and the Goblet of Fire (2005)
2                                Private Potter (1962)
3    Harry Potter and the Deathly Hallows: Part 2 (...
4                        Very Potter Musical, A (2009)
Name: title_movie, dtype: object
Top 5 book recommendations


0    Fantastic Beasts and Where to Find Them: The O...
1    Harry Potter and the Order of the Phoenix (Har...
2    Harry Potter and the Prisoner of Azkaban (Harr...
3              The Magician's Land (The Magicians, #3)
4                       Wicked Heart (Starcrossed, #3)
Name: title_book, dtype: object

In [278]:
get_global_reccs_cluster(suggested_film_ids)

Inputted films
0    Harry Potter and the Sorcerer's Stone (a.k.a. ...
1           Harry Potter and the Goblet of Fire (2005)
2                                Private Potter (1962)
3    Harry Potter and the Deathly Hallows: Part 2 (...
4                        Very Potter Musical, A (2009)
Name: title_movie, dtype: object
Top 5 book recommendations


0    Fantastic Beasts and Where to Find Them: The O...
1    Harry Potter and the Cursed Child - Parts One ...
2    Harry Potter and the Order of the Phoenix (Har...
3    Harry Potter and the Prisoner of Azkaban (Harr...
4              The Magician's Land (The Magicians, #3)
Name: title_book, dtype: object

In [280]:
# get_global_reccs_cluster_rating(suggested_film_ids)

### Frequent cluster

In [281]:
get_freq_cluster_reccs(suggested_film_ids)

Inputted films
0    Harry Potter and the Sorcerer's Stone (a.k.a. ...
1           Harry Potter and the Goblet of Fire (2005)
2                                Private Potter (1962)
3    Harry Potter and the Deathly Hallows: Part 2 (...
4                        Very Potter Musical, A (2009)
Name: title_movie, dtype: object
   clustering_label_bert  cluster_count
0                     13              2
1                     15              1
2                     29              1
3                     64              1
Top 5 book recommendations


0    Harry Potter and the Prisoner of Azkaban (Harr...
1    Fantastic Beasts and Where to Find Them: The O...
2    Harry Potter and the Order of the Phoenix (Har...
3    Harry Potter and the Deathly Hallows (Harry Po...
4    Harry Potter and the Sorcerer's Stone (Harry P...
Name: title_book, dtype: object

In [282]:
get_freq_cluster_reccs(cluster_film_ids)

Inputted films
0                                St George's Day (2012)
1                  10,000 Black Men Named George (2002)
2        Curious George: A Very Monkey Christmas (2009)
3                           George of the Jungle (1997)
4                    Madness of King George, The (1994)
5          George Carlin: Playin' with Your Head (1986)
6         George Carlin: Complaints & Grievances (2001)
7                         George of the Jungle 2 (2003)
8     Paul McCartney Really Is Dead: The Last Testam...
9                                 Curious George (2006)
10                               Come on George! (1939)
11              My Letter to George (Mesmerized) (1986)
12                          Journeys with George (2002)
13                  George Washington Slept Here (1942)
14                         Le roman de Georgette (2003)
15                 Killing of Sister George, The (1968)
16                  People vs. George Lucas, The (2010)
Name: title_movie, dtype: object


0    The Girl with a Clock for a Heart
1                       The Ice Dragon
2         Tinker, Tailor, Soldier, Spy
3                      The Book of Joe
4                      Beautiful Ruins
Name: title_book, dtype: object

# Identification of good book recommendation

In [283]:
def get_local_reccs_movies(user_books:list):
    
    verified_books = [book_id for book_id in user_books if book_id in X_all.item_id_book.tolist()]
    
    recommendations = pd.DataFrame(columns=["similarity","title_movie"])
    books = pd.DataFrame(verified_books,columns=["item_id_book"])
    books = pd.merge(books,X_all[["title_book","item_id_book"]],on="item_id_book",how="left")
    
    for book_id in verified_books:

        # Obtain vectors for user-inputted book
        book_cluster = X_vectors_books[X_vectors_books.index == book_id].clustering_label_bert.values[0]
        book_vector = X_vectors_books[X_vectors_books.index == book_id].drop(columns=["clustering_label_bert"])
        movies_vectors = X_vectors_movies[X_vectors_movies.clustering_label_bert == book_cluster].drop(columns=["clustering_label_bert"])

        # Calculate cosine similarity
        sim_movies = cosine_similarity(movies_vectors,book_vector)
 
        # Create summary table of movie with their similarity and relevant details
        sim_movies_detail = pd.DataFrame(sim_movies,index=movies_vectors.index,columns=["similarity"])
        sim_movies_detail = sim_movies_detail.sort_values("similarity",ascending=False)
        sim_movies_detail = pd.merge(sim_movies_detail,X_all[["title_movie","item_id_movie"]],
                                    on="item_id_movie",how="left")

        # Add top movie to recommendations dataframe
        top_movie = pd.DataFrame([sim_movies_detail.loc[0]])
        recommendations = pd.concat([recommendations,top_movie],axis=0, ignore_index=True)
    
    print("Inputted films")
    print(books)
    return recommendations["title_movie"]

In [284]:
def get_global_reccs_movies(user_books:list):
    
    verified_books = [book_id for book_id in user_books if book_id in X_all.item_id_book.tolist()]
    
    ## Collect vectors of all inputted books and calculate average vector
    books_id = pd.DataFrame(verified_books,columns=["item_id_book"])
    books_vectors = pd.merge(books_id,
                              X_vectors_books,
                              how="left",
                              on="item_id_book").set_index("item_id_book")
    avg_book_vector = pd.DataFrame([books_vectors.mean(numeric_only=True)]).drop(columns=["clustering_label_bert"])
    all_books_vectors = X_vectors_books.drop(columns=["clustering_label_bert"])
    
    ## Find cluster of nearest item (book)
    sim_books = cosine_similarity(all_books_vectors,avg_book_vector)
    sim_books_detail = pd.DataFrame(sim_books,
                                     index=all_books_vectors.index,
                                     columns=["similarity"]).sort_values("similarity",ascending=False).reset_index()
    closest_book_id = sim_books_detail.loc[0].item_id_book
    closest_cluster = X_vectors_books[X_vectors_books.index == closest_book_id].clustering_label_bert.values[0]
    movies_vectors = X_vectors_movies[X_vectors_movies.clustering_label_bert== closest_cluster].drop(columns=["clustering_label_bert"])

    ## Calculate cosine similarity
    sim_movies = cosine_similarity(movies_vectors,avg_book_vector)

    ## Create summary table of books with their similarity and relevant details
    sim_movies_detail = pd.DataFrame(sim_movies,index=movies_vectors.index,columns=["similarity"])
    sim_movies_detail = sim_movies_detail.sort_values("similarity",ascending=False)
    sim_movies_detail = pd.merge(sim_movies_detail,X_all[["title_movie","item_id_movie"]],on="item_id_movie",how="left")
    
    ## Take top 5 books and show results
    recommendations = sim_movies_detail.head(5)
    book_titles = pd.merge(books_id,X_all[["title_book","item_id_book"]],how="inner",on="item_id_book")
    print("Inputted books")
    print(book_titles.title_book)
    print ("Top 5 movie recommendations")
    return recommendations["title_movie"]

In [285]:
books = pd.DataFrame(X_all[X_all.is_movie==0].title_book).sort_values(by=["title_book"])
books = books.title_book.to_list()
# print(*books, sep='\n')

In [307]:
harry_potter = [book for book in books if "Potter" in book]
harry_potter

['Harry Potter Boxset (Harry Potter, #1-7)',
 'Harry Potter and the Chamber of Secrets (Harry Potter, #2)',
 'Harry Potter and the Cursed Child - Parts One and Two (Harry Potter, #8)',
 'Harry Potter and the Deathly Hallows (Harry Potter, #7)',
 'Harry Potter and the Half-Blood Prince (Harry Potter, #6)',
 'Harry Potter and the Methods of Rationality',
 'Harry Potter and the Order of the Phoenix (Harry Potter, #5)',
 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)',
 "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",
 'Harry Potter: The Prequel (Harry Potter, #0.5)',
 "James Potter and the Hall of Elders' Crossing (James Potter, #1)",
 'Short Stories from Hogwarts of Power, Politics and Pesky Poltergeists (Pottermore Presents, #2)']

In [315]:
other_books = [book for book in books if "treasure" in book]
other_books

[]

In [287]:
suggested_books = ["Treasure Island",
                   "Moby-Dick or, The Whale",
                   "Twenty Thousand Leagues Under the Sea"]

In [288]:
potter_books = ["Fantastic Beasts and Where to Find Them"]

In [289]:
suggested_book_ids = []

for book in harry_potter:
    if book in X_all.title_book.tolist():
        book_id = X_all[X_all.title_book == book].item_id_book.values[0]
        suggested_book_ids.append(book_id)

In [290]:
suggested_book_ids

[2962492.0,
 6231171.0,
 48765776.0,
 2963218.0,
 41335427.0,
 14911331.0,
 2809203.0,
 2402163.0,
 4640799.0,
 13810543.0,
 2556437.0,
 52218734.0]

In [291]:
get_local_reccs_movies(suggested_book_ids)

Inputted films
    item_id_book                                         title_book
0      2962492.0           Harry Potter Boxset (Harry Potter, #1-7)
1      6231171.0  Harry Potter and the Chamber of Secrets (Harry...
2     48765776.0  Harry Potter and the Cursed Child - Parts One ...
3      2963218.0  Harry Potter and the Deathly Hallows (Harry Po...
4     41335427.0  Harry Potter and the Half-Blood Prince (Harry ...
5     14911331.0        Harry Potter and the Methods of Rationality
6      2809203.0  Harry Potter and the Order of the Phoenix (Har...
7      2402163.0  Harry Potter and the Prisoner of Azkaban (Harr...
8      4640799.0  Harry Potter and the Sorcerer's Stone (Harry P...
9     13810543.0     Harry Potter: The Prequel (Harry Potter, #0.5)
10     2556437.0  James Potter and the Hall of Elders' Crossing ...
11    52218734.0  Short Stories from Hogwarts of Power, Politics...


0            Harry Potter and the Goblet of Fire (2005)
1     Harry Potter and the Deathly Hallows: Part 2 (...
2                               Maze Runner, The (2014)
3     Harry Potter and the Deathly Hallows: Part 2 (...
4     Harry Potter and the Deathly Hallows: Part 2 (...
5                            Jellyfish (Meduzot) (2007)
6            Harry Potter and the Goblet of Fire (2005)
7            Harry Potter and the Goblet of Fire (2005)
8     Harry Potter and the Deathly Hallows: Part 2 (...
9            Harry Potter and the Goblet of Fire (2005)
10                              Maze Runner, The (2014)
11           Harry Potter and the Goblet of Fire (2005)
Name: title_movie, dtype: object

In [292]:
get_global_reccs_movies(suggested_book_ids)

Inputted books
0              Harry Potter Boxset (Harry Potter, #1-7)
1     Harry Potter and the Chamber of Secrets (Harry...
2     Harry Potter and the Cursed Child - Parts One ...
3     Harry Potter and the Deathly Hallows (Harry Po...
4     Harry Potter and the Half-Blood Prince (Harry ...
5           Harry Potter and the Methods of Rationality
6     Harry Potter and the Order of the Phoenix (Har...
7     Harry Potter and the Prisoner of Azkaban (Harr...
8     Harry Potter and the Sorcerer's Stone (Harry P...
9        Harry Potter: The Prequel (Harry Potter, #0.5)
10    James Potter and the Hall of Elders' Crossing ...
11    Short Stories from Hogwarts of Power, Politics...
Name: title_book, dtype: object
Top 5 movie recommendations


0           Harry Potter and the Goblet of Fire (2005)
1    Harry Potter and the Deathly Hallows: Part 2 (...
2                                      Twitches (2005)
3                              Maze Runner, The (2014)
4                     Pillars of the Earth, The (2010)
Name: title_movie, dtype: object